**Package imports**

In [2]:
# Set up our Notebook with required packages
import pandas as pd
import altair as alt

## Altair has a default limit of 5000 rows for rendering charts (if our dataframe has more than this, we'll get an error when making the chart).
##  To override this limit, we can set the max_rows option.
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

# CC8: the Long-Run Prices Dataset

A notebook using the LRPD, described [here](https://cep.lse.ac.uk/pubs/download/occasional/op055.pdf).

</br></br></br></br>


**What is our goal?**

- Take a large dataset, extract and aggregate down to a level of data we can visualise.

Example workflow:
1. Load `prices` and `items` datasets (item description is separated from prices observations to reduce the size of the data files, we use a numerical `item_id` to match between them.)
2. Search the `items` dataset to find an item (or multiple items) that we want to analyse, record the `item_id` value(s).
3. Take the `item_id` value(s) and filter the `prices` dataframe

<br>

### Step 1. Loading data

#### 1a. Load prices dataset 

Note: the file linked to the cloud storage here is the same `db_prices` file available in the DropBox and linked in the lecture slides. Don't worry about the different `.parquet` file type here - it's just another file type like CSV & JSON, and works well with large datasets. The only change is to use `pd.read_parquet()` instead of `read_csv()` - the loaded dataframe is the same.

In [3]:
prices_df = pd.read_parquet('https://autocpi-public.s3.eu-west-2.amazonaws.com/lrpd/db_prices.parquet')

# What columns do we have?
print(prices_df.columns, "\n")

# Get summary statistics and round the results for better readability
prices_df.describe().round()

Index(['quote_date', 'shop_code', 'item_id_raw', 'region', 'price',
       'indicator_box', 'item_id'],
      dtype='object') 



,quote_date,shop_code,item_id_raw,region,price,item_id
count,48368958.0,48368958.0,48368973.0,48368958.0,48368958.0,48368973.0
mean,200776.0,477.0,388041.0,7.0,50.0,388398.0
std,1060.0,1532.0,146756.0,3.0,206.0,146672.0
min,198802.0,1.0,210101.0,1.0,0.0,210101.0
25%,199811.0,39.0,212917.0,3.0,1.0,212918.0
50%,200805.0,88.0,430128.0,7.0,5.0,430132.0
75%,201707.0,802.0,510406.0,9.0,20.0,510407.0
max,202510.0,20071.0,640406.0,13.0,44000.0,640406.0


View the dataframe

In [4]:
# Print 5 random rows from the dataframe
prices_df.sample(5)

,quote_date,shop_code,item_id_raw,region,price,indicator_box,item_id
44809456,200501.0,915.0,630105,8.0,79.989998,,630105
25013371,199607.0,179.0,430214,3.0,3.200000,S,430214
18743535,200508.0,85.0,310307,7.0,1.250000,,310307
30678013,200404.0,96.0,440233,13.0,5.000000,,440233
32759352,199608.0,115.0,510208,8.0,44.990002,,510208


To make visalising the data easier later, we can convert the `quote_date` column into the standard format. 
> We give the function the current date format, so 202510 is YYYYMM, which corresponds to `%Y%m` in date notation (`%Y`=YYYY, `%m`=MM). The notation is the same as formatting dates in Vega-Lite, and uses [D3 time formats](https://d3js.org/d3-time-format).

In [5]:
# Using pandas `to_datetime` function to convert 'quote_date' column to datetime format
prices_df['date'] = pd.to_datetime(prices_df['quote_date'], format='%Y%m')
prices_df.head(1)

,quote_date,shop_code,item_id_raw,region,price,indicator_box,item_id,date
0,200102.0,808.0,210101,12.0,0.35,Q,210101,2001-02-01


<br>
<br>
<br>

#### **1b.** Load items data

In [6]:
items_df = pd.read_parquet('https://autocpi-public.s3.eu-west-2.amazonaws.com/lrpd/db_item.parquet')

# Print the number of rows (i.e. products)
print(f"Number of rows in items_df: {len(items_df):,} \n")

items_df.head()

Number of rows in items_df: 1,387 



,item_id,description,date_quote_s,date_quote_e,n_obs
0,210101,LARGE LOAF-WHITE-SLICED-800G,198802,200401,36039
1,210102,LARGE LOAF-WHITE-UNSLICED-800G,198802,202510,56917
2,210105,LARGE WHOLEMEAL LOAF-UNSLICED,198802,200301,27161
3,210106,SIX BREAD ROLLS-WHITE/BROWN,198802,202510,67469
4,210107,"BROWN LOAF,400G,SLICED-GRAN",198903,200401,29361


The items data contains a description for each `item_id`, along with information on number of price observations, the earliest and last date that product has a price record.

There are 1,387 (as of Nov 2025) unique items in the long-run price dataset (keeping this out of the main prices dataset avoids unneccesarily duplicating this information for every price observation).

> NOTE: Statistical agencies update the basket of goods used to sample inflation as consumer spending habits change over time - often the result of technological change. This is why not all items have data up to today.

**Optional** Filter to only 'current' items, i.e. their latest observation matches 

In [7]:
# Filter items_df to only include rows where 'date_quote_e' is equal to the maximum value of 'date_quote_e'
# (By searching for the max, we don't need to hard-code a date, which may become outdated over time)
items_df_current = items_df[items_df['date_quote_e'] == items_df['date_quote_e'].max()].copy()
# same as items_df[items_df['date_quote_e'] == 202510]

print(f"Number of current items: {len(items_df_current):,} \n")
items_df_current.head()

Number of current items: 584 



,item_id,description,date_quote_s,date_quote_e,n_obs
1,210102,LARGE LOAF-WHITE-UNSLICED-800G,198802,202510,56917
3,210106,SIX BREAD ROLLS-WHITE/BROWN,198802,202510,67469
8,210111,WHITE SLICED LOAF BRANDED 750G,200402,202510,36848
10,210113,WHOLEMEAL SLICED LOAF BRANDED,200402,202510,35696
11,210114,CHILLED GARLIC BREAD,201002,202510,41672


<br>

#### **1c.** Searching for a product

Look in the items data for some products we're interested in.

In [8]:
# Print unique values in the 'description' column
items_df_current['description'].unique()

array(['LARGE LOAF-WHITE-UNSLICED-800G', 'SIX BREAD ROLLS-WHITE/BROWN',
       'WHITE SLICED LOAF BRANDED 750G', 'WHOLEMEAL SLICED LOAF BRANDED',
       'CHILLED GARLIC BREAD', 'WRAP / TORTILLA PACK 6-8',
       'GLUTEN FREE BREAD LF 300-550G', 'FLOUR-SELF-RAISING-1.5KG',
       'DRY SPAGHETTI OR PASTA 500G', 'CORN SNACK SINGLE PACK MAX 50G',
       'BASMATI RICE 500G-1KG', 'BREAKFAST CEREAL 1',
       'BREAKFAST CEREAL 2', 'CEREAL BAR', 'HOT OAT CEREAL',
       'RICE MICRO POUCH/TRAY 220-280G', 'BREAKFAST CEREAL GLUTEN FREE',
       'COUSCOUS PLAIN/FLAVOURED', 'RICE CAKES PACK 100-180G',
       'CREAM CRACKERS PACK 200G-300G', 'PLAIN BISCUITS-200-300G',
       'WHOLE SPONGE CAKE NOT FROZEN', 'PACK OF 5-6 INDIVIDUAL CAKES',
       'BISCUITS HALF CHOC 260-400G', 'CRUMPETS PACK 6-9 SPEC NUMBER',
       'HOME KILLED BEEF-LEAN MINCE KG', 'HOME KLD BEEF-RUMP/POPES STEAK',
       'FROZEN BEEFBURGERS PACK OF 4', 'BEEF ROASTING JOINT PER KG',
       'HK LAMB LOIN CHOP/STEAK PER KG', 'HOME KILL

<br>

### Step 2. Aggregating

Before plotting, we need to filter aggregate the data.

#### 2a. Filter prices on `item_id`.


<br>

How do we filter?
`df[df['columnx'] == 'xyz']`:
- `df['columnx'] == 'xyz'` -> Call a column and test against a condition (this returns a column of true/false values)
- `df[...]` -> Wrap the test with the main dataframe to filter to rows where our condition is true.

To help searching for items without scrolling back to top

In [35]:
# If we add '|' to the string, it acts as an OR operator, so we can search for multiple keywords at once.
items_df_current[items_df_current['description'].str.contains('bread', case=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
3,210106,SIX BREAD ROLLS-WHITE/BROWN,198802,202510,67469
11,210114,CHILLED GARLIC BREAD,201002,202510,41672
13,210116,GLUTEN FREE BREAD LF 300-550G,202402,202510,2205
126,211211,FROZEN BREAD/BAT FISH 400-550G,201002,202510,40004


In [28]:
items_df_current[items_df_current['description'].str.contains('frozen', case=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
43,210320,WHOLE SPONGE CAKE NOT FROZEN,200002,202510,39549
53,210414,FROZEN BEEFBURGERS PACK OF 4,199202,202510,50005
118,211106,FROZEN PRAWNS PER KG,200402,202510,36945
125,211210,FROZEN FISH FINGERS 8-12 PK,200402,202510,53776
126,211211,FROZEN BREAD/BAT FISH 400-550G,201002,202510,40004
246,212405,FROZEN CHIPS 900G-1.5KG,199202,202510,89845
288,212609,FROZEN GARDEN PEAS 800G-1KG,199402,202510,61860
291,212612,FROZEN PRE-PREPARED VEGTABLES,202102,202510,17685
332,212809,BERRIES FROZEN PACK,202302,202510,10118
367,212943,YORKSHIRE PUDDING FROZEN,202202,202510,9752


In [25]:
items_df_current[items_df_current['description'].str.contains('chicken', case=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
77,210905,FRESH/CHILLED CHICKEN PER KG,198903,202510,105251
82,210910,FRESH BONELESS CHICKEN BREAST,200302,202510,65366
103,211019,FROZ CHICKEN NUGGETS 220-600G,200502,202510,28024
109,211026,CHICKEN KIEV 2 PACK 240-325G,201502,202510,26070
427,220326,TAKEAWAY CHICKEN & CHIPS,201202,202510,19909


In [51]:
items_df_current[items_df_current['description'].str.contains('beef', case=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
48,210403,HOME KILLED BEEF-LEAN MINCE KG,198802,202510,121745
50,210406,HOME KLD BEEF-RUMP/POPES STEAK,198802,202510,117178
53,210414,FROZEN BEEFBURGERS PACK OF 4,199202,202510,50005
55,210416,BEEF ROASTING JOINT PER KG,202002,202510,12885


In [ ]:
# 210106 Bread, 211210 frozen fish, 212405 frozen chips, 210905 Fresh chicken, 210406 Beef Steak

In [11]:
# 1. Filter prices based on TWO item IDs. 
# (we can use `.isin()` to filter by checking against a multiple values in list)
takeaway_prices = prices_df[prices_df['item_id'].isin([210106, 211210, 212405, 210905, 210403])].copy()

# 2. Group by date and item_id, calculate mean price, store in new dataframe
takeaway_price_stats = takeaway_prices.groupby(['date', 'item_id']).agg({'price': 'mean'}).reset_index()

# We have columns 'date', 'item_id', 'price'
takeaway_price_stats

,date,item_id,price
0,1988-02-01,210106,0.477151
1,1988-02-01,210403,2.806472
2,1988-03-01,210106,0.469620
3,1988-03-01,210403,2.891031
4,1988-04-01,210106,0.474181
...,...,...,...
2003,2025-10-01,210106,1.272340
2004,2025-10-01,210403,11.601636
2005,2025-10-01,210905,4.610573
2006,2025-10-01,211210,3.027621


**Visualise**

In [12]:
alt.Chart(takeaway_price_stats).mark_line().encode(
    x=alt.X("date:T"),
    y=alt.Y("price"),
    color=alt.Color("item_id:N")
).properties(
    width=400,
    height=250,
    title=alt.TitleParams(
        text="Takeaway Items: Price History",
        subtitle=["Mean average price", "Source: ONS microdata via Davies (2021)"],
        anchor="start",
        frame='group'
    )
)

alt.Chart(...)

<br>

**How do we add replace our item_id with the descriptions?**
We could manually set the values in the chart spec, but better to either:
1. Merge the `description` from items_df into our filtered prices
2. Set our own values for each item_id. And add new column mapping item_id to some description we've set.

In [13]:
# Option 1. Call `.merge` method on our filtered dataframe to add descriptions. 
#  match `on` item_id column in each dataframe. `how='left'` to keep all rows in takeaway_price_stats.
takeaway_price_stats.merge(items_df[['item_id', 'description']], on='item_id', how='left')

,date,item_id,price,description
0,1988-02-01,210106,0.477151,SIX BREAD ROLLS-WHITE/BROWN
1,1988-02-01,210403,2.806472,HOME KILLED BEEF-LEAN MINCE KG
2,1988-03-01,210106,0.469620,SIX BREAD ROLLS-WHITE/BROWN
3,1988-03-01,210403,2.891031,HOME KILLED BEEF-LEAN MINCE KG
4,1988-04-01,210106,0.474181,SIX BREAD ROLLS-WHITE/BROWN
...,...,...,...,...
2003,2025-10-01,210106,1.272340,SIX BREAD ROLLS-WHITE/BROWN
2004,2025-10-01,210403,11.601636,HOME KILLED BEEF-LEAN MINCE KG
2005,2025-10-01,210905,4.610573,FRESH/CHILLED CHICKEN PER KG
2006,2025-10-01,211210,3.027621,FROZEN FISH FINGERS 8-12 PK


In [14]:
# Option 2. Define our own mapping of item_id to description, and add new column.
item_labels = {210106: 'Bread', 211210: 'frozen fish', 212405: 'frozen chips', 210905: 'Fresh chicken', 210403: 'Beef'} 

# Use the `.map()` method to create new column 'label' based on mapping dictionary. It finds the value in the dictionary for each item_id.
takeaway_price_stats['label'] = takeaway_price_stats['item_id'].map(item_labels)
takeaway_price_stats

,date,item_id,price,label
0,1988-02-01,210106,0.477151,Bread
1,1988-02-01,210403,2.806472,Beef
2,1988-03-01,210106,0.469620,Bread
3,1988-03-01,210403,2.891031,Beef
4,1988-04-01,210106,0.474181,Bread
...,...,...,...,...
2003,2025-10-01,210106,1.272340,Bread
2004,2025-10-01,210403,11.601636,Beef
2005,2025-10-01,210905,4.610573,Fresh chicken
2006,2025-10-01,211210,3.027621,frozen fish


<br>

Visualise again, encoding colour on our new label / description column

In [19]:
alt.Chart(takeaway_price_stats).mark_line().encode(
    x=alt.X("date:T"),
    y=alt.Y("price"),
    color=alt.Color("label:N")
).properties(
    width=400,
    height=250,
    title=alt.TitleParams(
        text="Food products I usually buy",
        subtitle=["Mean average monthly price", "Source: ONS microdata via Davies (2021)"],
        anchor="start",
        frame='group'
    )
)

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [29]:
takeaway_price_stats_2005 = takeaway_price_stats[
    takeaway_price_stats["date"] >= "2005-01-01"
]

In [31]:
takeaway_price_stats_2005["date"] = pd.to_datetime(takeaway_price_stats_2005["date"])

/var/folders/p7/_ml_f8590cg3dmhph8rb7ydc0000gn/T/ipykernel_36495/3206033494.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  takeaway_price_stats_2005["date"] = pd.to_datetime(takeaway_price_stats_2005["date"])


In [36]:
lines = alt.Chart(takeaway_price_stats_2005).mark_line().encode(
    x=alt.X("date:T")
        .title('')
        .axis(grid=False),
    y=alt.Y("price:Q")
        .title('')
        .axis(labelExpr="'£' + datum.value"),
    color=alt.Color("label:N")
        .legend(None)
        .scale(
            range=[
                '#70B0FA',
                '#F54927',
                '#2FBF71',
                '#F2C94C',
                '#9B51E0'
            ]
        )
).properties(
    width=400,
    height=250,
    title=alt.TitleParams(
        text="Food products I usually buy",
        subtitle=[
            "Mean average monthly price",
            "Source: ONS microdata via Davies (2021)"
        ],
        anchor="start",
        frame='group'
    )
)

text = lines.mark_text(
    align='left',
    dx=5,
    fontSize=12,
    fontWeight='bold'
).encode(
    x=alt.X("date:T").aggregate('max'),
    y=alt.Y("price:Q").aggregate({'argmax': 'date'}),
    text=alt.Text('label:N')
)

chart = (lines + text).configure_view(
    strokeWidth=0
)

chart


/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [ ]:
#Exporting CSV to do the chart in vegalite 

takeaway_price_stats_2005.to_csv('takeaway_price_stats_2005.csv', index=False)

### Chart 2

In [92]:
items_df_current[items_df_current['description'].str.contains('women', case=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
862,510206,WOMENS BLOUSE/SHIRT 1,198802,202510,123005
864,510208,WOMEN'S FORMAL JACKET,198802,202510,121087
868,510212,WOMEN'S JUMPER,198802,202510,123638
871,510215,WOMENS BASIC PLAIN T-SHIRT,198802,202510,102756
873,510219,WOMEN'S CASUAL OUTER JACKET,199702,202510,92647
877,510223,WOMEN'S LONG SLV TOP-NT BLOUSE,199702,202510,92645
887,510233,WOMEN'S CARDIGAN,199902,202510,89806
889,510235,WOMEN'S TROUSERS-CASUAL,200102,202510,88024
890,510236,WOMEN'S TROUSERS-FORMAL,200102,202510,89872
891,510237,WOMEN'S SKIRT: CASUAL,200102,202510,83349


In [47]:
items_df_current[items_df_current['description'].str.contains('jacket', case=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
820,510104,MENS JACKET/COAT,198802,202510,122442
848,510134,MENS FORMAL JACKET/BLAZER,202202,202510,13015
864,510208,WOMEN'S FORMAL JACKET,198802,202510,121087
873,510219,WOMEN'S CASUAL OUTER JACKET,199702,202510,92647
907,510256,WOMEN'S SHOWERPROOF JACKET,200602,202510,34933
959,510347,GIRLS JACKET (5-13 YEARS),201102,202510,55706


In [48]:
items_df_current[items_df_current['description'].str.contains('jumper', case=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
824,510108,MEN'S JUMPER-KNITTED,198802,202510,124502
834,510120,MEN'S JUMPER OR SWEATSHIRT,199702,202510,98703
868,510212,WOMEN'S JUMPER,198802,202510,123638
962,510350,BOY'S JUMPER/SWTSHRT/CARDIGAN,202102,202510,19484


In [50]:
items_df_current[items_df_current['description'].str.contains('trousers', case=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
831,510117,MEN'S FORMAL RDY-MADE TROUSERS,199702,202510,104919
832,510118,MEN'S CASUAL TROUSERS,199702,202510,102736
889,510235,WOMEN'S TROUSERS-CASUAL,200102,202510,88024
890,510236,WOMEN'S TROUSERS-FORMAL,200102,202510,89872
940,510324,TROUSERS FOR SCHOOL 5-13 YRS,200202,202510,77162
956,510344,GIRLS TROUSERS NOT DENIM 5-13,200602,202510,66471
963,510351,INFANT'S TROUSERS 0-2 YEARS,202102,202510,15709


In [90]:
items_df_current[items_df_current['description'].str.contains('sock', case=False)]

,item_id,description,date_quote_s,date_quote_e,n_obs
978,510413,MENS SOCKS 1 PAIR,198802,202510,133027
995,510438,CHILDS SOCKS ONE PAIR UK9,202202,202510,14689
997,510440,WOMENS SOCK 1 PAIR,202402,202510,6349


In [55]:
gender_prices = prices_df[prices_df['item_id'].isin([510104, 510208, 510108, 510212, 510118, 510235, 510120, 510261])].copy()

# 2. Group by date and item_id, calculate mean price, store in new dataframe
gender_prices_stat = gender_prices.groupby(['date', 'item_id']).agg({'price': 'mean'}).reset_index()

# We have columns 'date', 'item_id', 'price'
gender_prices_stat

,date,item_id,price
0,1988-02-01,510104,33.625988
1,1988-02-01,510108,16.794111
2,1988-02-01,510208,39.951195
3,1988-02-01,510212,15.182670
4,1988-03-01,510104,33.470524
...,...,...,...
2898,2025-10-01,510120,43.324360
2899,2025-10-01,510208,66.601006
2900,2025-10-01,510212,37.775963
2901,2025-10-01,510235,32.214405


In [58]:
gender_prices_stat.merge(items_df[['item_id', 'description']], on='item_id', how='left')

,date,item_id,price,description
0,1988-02-01,510104,33.625988,MENS JACKET/COAT
1,1988-02-01,510108,16.794111,MEN'S JUMPER-KNITTED
2,1988-02-01,510208,39.951195,WOMEN'S FORMAL JACKET
3,1988-02-01,510212,15.182670,WOMEN'S JUMPER
4,1988-03-01,510104,33.470524,MENS JACKET/COAT
...,...,...,...,...
2898,2025-10-01,510120,43.324360,MEN'S JUMPER OR SWEATSHIRT
2899,2025-10-01,510208,66.601006,WOMEN'S FORMAL JACKET
2900,2025-10-01,510212,37.775963,WOMEN'S JUMPER
2901,2025-10-01,510235,32.214405,WOMEN'S TROUSERS-CASUAL


In [66]:
# Option 2. Define our own mapping of item_id to description, and add new column.
gender_item_labels = {510104: 'Men jacket', 510208: 'Women jacket', 510108: 'Men jumper', 510212: 'Women jumper', 510118: 'Men trousers', 510235: 'Women trousers', 510120: 'Men sweatshirt', 510261: 'Women sweatshirt'} 

# Use the `.map()` method to create new column 'label' based on mapping dictionary. It finds the value in the dictionary for each item_id.
gender_prices_stat['label'] = gender_prices_stat['item_id'].map(gender_item_labels)
gender_prices_stat

,date,item_id,price,label,gender,product
0,1988-02-01,510104,33.625988,Men jacket,Men,Jacket
1,1988-02-01,510108,16.794111,Men jumper,Men,Jumper
2,1988-02-01,510208,39.951195,Women jacket,Women,Jacket
3,1988-02-01,510212,15.182670,Women jumper,Women,Jumper
4,1988-03-01,510104,33.470524,Men jacket,Men,Jacket
...,...,...,...,...,...,...
2898,2025-10-01,510120,43.324360,Men sweatshirt,Men,Sweatshirt
2899,2025-10-01,510208,66.601006,Women jacket,Women,Jacket
2900,2025-10-01,510212,37.775963,Women jumper,Women,Jumper
2901,2025-10-01,510235,32.214405,Women trousers,Women,Trousers


In [64]:
# Option 2. Define our own mapping of item_id to description, and add new column.
gender_item = {510104: 'Men', 510208: 'Women', 510108: 'Men', 510212: 'Women', 510118: 'Men', 510235: 'Women', 510120: 'Men', 510261: 'Women'} 

# Use the `.map()` method to create new column 'label' based on mapping dictionary. It finds the value in the dictionary for each item_id.
gender_prices_stat['gender'] = gender_prices_stat['item_id'].map(gender_item)
gender_prices_stat

,date,item_id,price,label,gender
0,1988-02-01,510104,33.625988,Men jacket,Men
1,1988-02-01,510108,16.794111,Men jumper,Men
2,1988-02-01,510208,39.951195,Women jacket,Women
3,1988-02-01,510212,15.182670,Women jumper,Women
4,1988-03-01,510104,33.470524,Men jacket,Men
...,...,...,...,...,...
2898,2025-10-01,510120,43.324360,Men sweatshirt,Men
2899,2025-10-01,510208,66.601006,Women jacket,Women
2900,2025-10-01,510212,37.775963,Women jumper,Women
2901,2025-10-01,510235,32.214405,Women trousers,Women


In [84]:
# Option 2. Define our own mapping of item_id to description, and add new column.
product_item = {510104: 'Jacket', 510208: 'Jacket', 510108: 'Jumper', 510212: 'Jumper', 510118: 'Trousers', 510235: 'Trousers', 510120: 'Sweatshirt', 510261: 'Sweatshirt'} 

# Use the `.map()` method to create new column 'label' based on mapping dictionary. It finds the value in the dictionary for each item_id.
gender_prices_stat['product'] = gender_prices_stat['item_id'].map(product_item)
gender_prices_stat

,date,item_id,price,label,gender,product
0,1988-02-01,510104,33.625988,Men jacket,Men,Jacket
1,1988-02-01,510108,16.794111,Men jumper,Men,Jumper
2,1988-02-01,510208,39.951195,Women jacket,Women,Jacket
3,1988-02-01,510212,15.182670,Women jumper,Women,Jumper
4,1988-03-01,510104,33.470524,Men jacket,Men,Jacket
...,...,...,...,...,...,...
2898,2025-10-01,510120,43.324360,Men sweatshirt,Men,Sweatshirt
2899,2025-10-01,510208,66.601006,Women jacket,Women,Jacket
2900,2025-10-01,510212,37.775963,Women jumper,Women,Jumper
2901,2025-10-01,510235,32.214405,Women trousers,Women,Trousers


In [85]:
# Option 2. Define our own mapping of item_id to description, and add new column.
description_item = {510104: "MENS JACKET/COAT", 510208: "WOMEN'S FORMAL JACKET", 510108: "MEN'S JUMPER-KNITTED", 510212: "WOMEN'S JUMPER", 510118: "MEN'S CASUAL TROUSERS", 510235: "WOMEN'S TROUSERS-CASUAL", 510120: "MEN'S JUMPER OR SWEATSHIRT", 510261: "WOMEN'S SWEATSHIRT"} 

# Use the `.map()` method to create new column 'label' based on mapping dictionary. It finds the value in the dictionary for each item_id.
gender_prices_stat['description'] = gender_prices_stat['item_id'].map(description_item)
gender_prices_stat

,date,item_id,price,label,gender,product,description
0,1988-02-01,510104,33.625988,Men jacket,Men,Jacket,MENS JACKET/COAT
1,1988-02-01,510108,16.794111,Men jumper,Men,Jumper,MEN'S JUMPER-KNITTED
2,1988-02-01,510208,39.951195,Women jacket,Women,Jacket,WOMEN'S FORMAL JACKET
3,1988-02-01,510212,15.182670,Women jumper,Women,Jumper,WOMEN'S JUMPER
4,1988-03-01,510104,33.470524,Men jacket,Men,Jacket,MENS JACKET/COAT
...,...,...,...,...,...,...,...
2898,2025-10-01,510120,43.324360,Men sweatshirt,Men,Sweatshirt,MEN'S JUMPER OR SWEATSHIRT
2899,2025-10-01,510208,66.601006,Women jacket,Women,Jacket,WOMEN'S FORMAL JACKET
2900,2025-10-01,510212,37.775963,Women jumper,Women,Jumper,WOMEN'S JUMPER
2901,2025-10-01,510235,32.214405,Women trousers,Women,Trousers,WOMEN'S TROUSERS-CASUAL


In [60]:
alt.Chart(gender_prices_stat).mark_line().encode(
    x=alt.X("date:T"),
    y=alt.Y("price"),
    color=alt.Color("label:N")
).properties(
    width=400,
    height=250,
    title=alt.TitleParams(
        text="Food products I usually buy",
        subtitle=["Mean average monthly price", "Source: ONS microdata via Davies (2021)"],
        anchor="start",
        frame='group'
    )
)

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [86]:
gender_prices_stat_2000 = gender_prices_stat[
    gender_prices_stat["date"] >= "2000-01-01"
]

In [87]:
gender_prices_stat_2000["date"] = pd.to_datetime(gender_prices_stat_2000["date"])

/var/folders/p7/_ml_f8590cg3dmhph8rb7ydc0000gn/T/ipykernel_36495/3069960139.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_prices_stat_2000["date"] = pd.to_datetime(gender_prices_stat_2000["date"])


In [88]:
#Exporting CSV to do the chart in vegalite 

gender_prices_stat_2000.to_csv('gender_prices.csv', index=False)

In [89]:
gender_prices_stat_2000

,date,item_id,price,label,gender,product,description
689,2000-01-01,510104,53.617294,Men jacket,Men,Jacket,MENS JACKET/COAT
690,2000-01-01,510108,26.001524,Men jumper,Men,Jumper,MEN'S JUMPER-KNITTED
691,2000-01-01,510118,27.944092,Men trousers,Men,Trousers,MEN'S CASUAL TROUSERS
692,2000-01-01,510120,25.273527,Men sweatshirt,Men,Sweatshirt,MEN'S JUMPER OR SWEATSHIRT
693,2000-01-01,510208,57.207096,Women jacket,Women,Jacket,WOMEN'S FORMAL JACKET
...,...,...,...,...,...,...,...
2898,2025-10-01,510120,43.324360,Men sweatshirt,Men,Sweatshirt,MEN'S JUMPER OR SWEATSHIRT
2899,2025-10-01,510208,66.601006,Women jacket,Women,Jacket,WOMEN'S FORMAL JACKET
2900,2025-10-01,510212,37.775963,Women jumper,Women,Jumper,WOMEN'S JUMPER
2901,2025-10-01,510235,32.214405,Women trousers,Women,Trousers,WOMEN'S TROUSERS-CASUAL
